In [8]:
import os
import numpy as np 
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [9]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/pdf-db/tsla-20231231-gen.pdf
/kaggle/input/pdf-db/goog-10-k-2023.pdf
/kaggle/input/pdf-db/uber-10-k-2023.pdf


In [10]:
!nvidia-smi

Wed Nov 13 19:35:12 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8             12W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch --progress-bar off
!pip install -qqq transformers --progress-bar off
!pip install -qqq langchain --progress-bar off
!pip install -qqq chromadb --progress-bar off
!pip install -qqq pypdf --progress-bar off
!pip install -qqq xformers --progress-bar off
!pip install -qqq sentence_transformers --progress-bar off
!pip install -qqq InstructorEmbedding --progress-bar off
!pip install -qqq pdf2image --progress-bar off

In [12]:
!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.1/auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl

In [13]:
!pip install -qqq auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl --progress-bar off

In [15]:
!sudo apt-get install poppler-utils -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpoppler118 poppler-data
Suggested packages:
  ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  libpoppler118 poppler-data poppler-utils
0 upgraded, 3 newly installed, 0 to remove and 68 not upgraded.
Need to get 3427 kB of archives.
After this operation, 17.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 poppler-data all 0.4.11-1 [2171 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler118 amd64 22.02.0-2ubuntu0.5 [1071 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 3427 kB in 1s (2697 kB/s)       
Selecting previously unselected

In [16]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [17]:
pdf_directory = "/kaggle/input/pdf-db"
loader = PyPDFDirectoryLoader(pdf_directory)
docs = loader.load()

In [18]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [19]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

1713

In [20]:
persist_directory = "/kaggle/working/db"

if not os.path.exists(persist_directory):
    print("Creating Chroma database at:", persist_directory)
    db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory)
else:
    print("Loading existing Chroma database from:", persist_directory)
    db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

Creating Chroma database at: /kaggle/working/db


In [26]:
# Check if GPU is available
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

# Define device map for GPU setup (if using multiple GPUs, adjust accordingly)
device_map = "auto"  # This automatically assigns devices in a multi-GPU setup. Use "cuda:0" for a specific GPU.

# Load the model
model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,  # Ensure device is passed
    device_map=device_map,  # This will allow automatic device assignment
    quantize_config=None,
)

  0%|          | 0/1523 [00:00<?, ?w/s]

In [27]:

DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()

In [28]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [29]:

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

In [30]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [31]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [32]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [33]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [34]:
result = qa_chain("What are the risk factors associated with Google and Tesla?")
print(result)

 I can provide information on the risk factors associated with Tesla, as described in the provided text. However, I cannot provide information on the risk factors associated with Google, as there is no mention of Google in the text.

According to the text, the risk factors associated with Tesla include:

1. Competition for skilled individuals with automotive or technology experience, particularly in regions where Tesla has or will have operations.
2. Strong competition from mature and prosperous companies with far greater financial resources than Tesla, as well as start-ups and emerging companies that promise short-term growth opportunities.
3. The emphasis on equity-based awards and benefits in Tesla's compensation philosophy, which reflects its startup origins.
4. Negative publicity related to Tesla, which may cause employees to leave the company or choose other employers over Tesla.
5. Various factors that may harm Tesla's business, financial condition, and operating results, such a

In [40]:
result = qa_chain("What is the total revenue for Google Search?")
for k,v in result.items():
    if k=="result":
        print(v)

 Based on the information provided in the context, the total revenue for Google Search is not explicitly stated. However, we can calculate it based on the information given.

In 2021, the revenues from Google Search & other were $148,951 million.

In 2022, the revenues from Google Search & other were $162,450 million.

In 2023, the revenues from Google Search & other were $175,033 million.

Therefore, the total revenue for Google Search from 2021 to 2023 is:

$148,951 + $162,450 + $175,033 = $486,434 million.

So, the total revenue for Google Search is $486,434 million.
  Based on the information provided in the context, the total revenue for Google Search is not explicitly stated. However, we can calculate it based on the information given.

In 2021, the revenues from Google Search & other were $148,951 million.

In 2022, the revenues from Google Search & other were $162,450 million.

In 2023, the revenues from Google Search & other were $175,033 million.

Therefore, the total revenue

In [44]:
result = qa_chain("What is main business of Uber?")
for k,v in result.items():
    if k=="result":
        print(v)

 Based on the information provided in the context, the main business of Uber is to develop and operate a technology platform that connects consumers with providers of ride services, meal preparation, grocery, and other delivery services, as well as with public transportation networks.
  Based on the information provided in the context, the main business of Uber is to develop and operate a technology platform that connects consumers with providers of ride services, meal preparation, grocery, and other delivery services, as well as with public transportation networks.
